In [1]:
import pandas as pd
import numpy as np
import torch
import sklearn
from sklearn.metrics import *
from sklearn.model_selection import *
import dgl
import networkx as nx
import torchmetrics

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
dataset = pd.read_csv('creditcard.csv')

In [4]:
dataset = dataset.reset_index()
dataset = dataset.rename(columns={'Time':'destination','index':'source'})

In [5]:
train,test = train_test_split(dataset,test_size=0.2,stratify=dataset.Class,random_state=2)

In [6]:
G = nx.Graph()
G = nx.from_pandas_edgelist(dataset,source='source',target='destination')
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 284807
Number of edges: 284807
Average degree:   2.0000


In [7]:
nodes_data = torch.tensor(dataset.source.values).long()
edges_data = torch.tensor(dataset.destination.values).long()
nodes_features = torch.tensor(dataset.drop(columns=['source','destination','Class']).values).float()
nodes_labels = torch.tensor(dataset.Class.values).long()

In [8]:
graph = dgl.graph((nodes_data,edges_data),num_nodes=nodes_data.shape[0])

In [9]:
graph.ndata['feat'] = nodes_features
graph.ndata['label'] = nodes_labels

In [10]:
train_mask = torch.zeros(nodes_data.shape[0],dtype=torch.bool)
test_mask = torch.zeros(nodes_data.shape[0],dtype=torch.bool)

In [11]:
train_md = torch.tensor(train.reset_index()['index'].values).long()
test_md = torch.tensor(test.reset_index()['index'].values).long()

In [12]:
train_mask[train_md] = True
test_mask[test_md] = True

In [13]:
graph.ndata['train_mask'] = train_mask
graph.ndata['test_mask'] = test_mask

In [14]:
print(graph)

Graph(num_nodes=284807, num_edges=284807,
      ndata_schemes={'feat': Scheme(shape=(29,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})


In [15]:
class GCN(torch.nn.Module):
    def __init__(self,n_inputs):
        super(GCN, self).__init__()
        self.conv1 = dgl.nn.SAGEConv(n_inputs,12,aggregator_type='pool')
        self.norm1 = torch.nn.BatchNorm1d(12)
        self.conv2 = dgl.nn.SAGEConv(12,5,aggregator_type='pool')
        self.norm2 = torch.nn.BatchNorm1d(5)
        self.hidden1 = torch.nn.Linear(5,2)
        torch.nn.init.xavier_normal_(self.hidden1.weight)

    def forward(self, g, n_input):
        X = self.conv1(g, n_input)
        X = torch.nn.functional.selu(self.norm1(X))
        X = self.conv2(g, X)
        X = torch.nn.functional.selu(self.norm2(X))
        X = self.hidden1(X)
        X = torch.nn.functional.logsigmoid(X)
        return X

In [16]:
model = GCN(graph.ndata['feat'].shape[1])

In [17]:
model

GCN(
  (conv1): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_pool): Linear(in_features=29, out_features=29, bias=True)
    (fc_self): Linear(in_features=29, out_features=12, bias=False)
    (fc_neigh): Linear(in_features=29, out_features=12, bias=False)
  )
  (norm1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): SAGEConv(
    (feat_drop): Dropout(p=0.0, inplace=False)
    (fc_pool): Linear(in_features=12, out_features=12, bias=True)
    (fc_self): Linear(in_features=12, out_features=5, bias=False)
    (fc_neigh): Linear(in_features=12, out_features=5, bias=False)
  )
  (norm2): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hidden1): Linear(in_features=5, out_features=2, bias=True)
)

In [18]:
# class_weights=sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(dataset.Class),y=dataset.Class)
# class_weights=torch.tensor(class_weights,dtype=torch.float32)
# class_weights

In [19]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.003)

In [20]:
print('--------------------------------------------------------------------------------------------------------------')

for epoch in range(1,1000):
    
    model.train()
    optimizer.zero_grad()
    yhat_train = model.forward(graph,graph.ndata['feat'])
    loss_train = criterion(yhat_train[graph.ndata['train_mask']],graph.ndata['label'][graph.ndata['train_mask']])
    loss_train.backward()
    optimizer.step()
    
    model.eval()
    loss_test = criterion(yhat_train[graph.ndata['test_mask']],graph.ndata['label'][graph.ndata['test_mask']])
    predictions = torch.max(yhat_train[graph.ndata['test_mask']],1)[1].detach().numpy()
    
    probabilities = torch.max(yhat_train[graph.ndata['test_mask']],1)[0]
    actuals = graph.ndata['label'][graph.ndata['test_mask']]
    
    accuracy = accuracy_score(actuals,predictions)
    precision = precision_score(actuals,predictions)
    recall = recall_score(actuals,predictions)
    print('Epoch: {}, Loss train: {:.4f}, Loss test: {:.4f}, Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}'.format(
            epoch,loss_train,loss_test,accuracy,precision,recall))
    print('--------------------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------------------
Epoch: 1, Loss train: 1.1348, Loss test: 1.1344, Accuracy: 0.1970, Precision: 0.0017, Recall: 0.7857
--------------------------------------------------------------------------------------------------------------
Epoch: 2, Loss train: 1.1051, Loss test: 1.1047, Accuracy: 0.2005, Precision: 0.0017, Recall: 0.7959
--------------------------------------------------------------------------------------------------------------
Epoch: 3, Loss train: 1.0752, Loss test: 1.0748, Accuracy: 0.2054, Precision: 0.0017, Recall: 0.8061
--------------------------------------------------------------------------------------------------------------
Epoch: 4, Loss train: 1.0453, Loss test: 1.0450, Accuracy: 0.2094, Precision: 0.0018, Recall: 0.8061
--------------------------------------------------------------------------------------------------------------
Epoch: 5, Loss train: 1.0158, Loss test: 

Epoch: 39, Loss train: 0.5800, Loss test: 0.5802, Accuracy: 0.8148, Precision: 0.0044, Recall: 0.4694
--------------------------------------------------------------------------------------------------------------
Epoch: 40, Loss train: 0.5747, Loss test: 0.5749, Accuracy: 0.8166, Precision: 0.0044, Recall: 0.4694
--------------------------------------------------------------------------------------------------------------
Epoch: 41, Loss train: 0.5694, Loss test: 0.5696, Accuracy: 0.8180, Precision: 0.0044, Recall: 0.4694
--------------------------------------------------------------------------------------------------------------
Epoch: 42, Loss train: 0.5642, Loss test: 0.5643, Accuracy: 0.8196, Precision: 0.0045, Recall: 0.4694
--------------------------------------------------------------------------------------------------------------
Epoch: 43, Loss train: 0.5589, Loss test: 0.5591, Accuracy: 0.8217, Precision: 0.0044, Recall: 0.4592
----------------------------------------------

Epoch: 78, Loss train: 0.3857, Loss test: 0.3854, Accuracy: 0.8908, Precision: 0.0069, Recall: 0.4388
--------------------------------------------------------------------------------------------------------------
Epoch: 79, Loss train: 0.3813, Loss test: 0.3811, Accuracy: 0.8920, Precision: 0.0070, Recall: 0.4388
--------------------------------------------------------------------------------------------------------------
Epoch: 80, Loss train: 0.3771, Loss test: 0.3768, Accuracy: 0.8935, Precision: 0.0071, Recall: 0.4388
--------------------------------------------------------------------------------------------------------------
Epoch: 81, Loss train: 0.3728, Loss test: 0.3726, Accuracy: 0.8952, Precision: 0.0072, Recall: 0.4388
--------------------------------------------------------------------------------------------------------------
Epoch: 82, Loss train: 0.3687, Loss test: 0.3685, Accuracy: 0.8966, Precision: 0.0073, Recall: 0.4388
----------------------------------------------

Epoch: 117, Loss train: 0.2508, Loss test: 0.2507, Accuracy: 0.9692, Precision: 0.0274, Recall: 0.4898
--------------------------------------------------------------------------------------------------------------
Epoch: 118, Loss train: 0.2481, Loss test: 0.2480, Accuracy: 0.9702, Precision: 0.0283, Recall: 0.4898
--------------------------------------------------------------------------------------------------------------
Epoch: 119, Loss train: 0.2455, Loss test: 0.2454, Accuracy: 0.9710, Precision: 0.0297, Recall: 0.5000
--------------------------------------------------------------------------------------------------------------
Epoch: 120, Loss train: 0.2429, Loss test: 0.2428, Accuracy: 0.9721, Precision: 0.0308, Recall: 0.5000
--------------------------------------------------------------------------------------------------------------
Epoch: 121, Loss train: 0.2403, Loss test: 0.2402, Accuracy: 0.9730, Precision: 0.0312, Recall: 0.4898
-----------------------------------------

Epoch: 156, Loss train: 0.1666, Loss test: 0.1667, Accuracy: 0.9905, Precision: 0.1034, Recall: 0.5918
--------------------------------------------------------------------------------------------------------------
Epoch: 157, Loss train: 0.1648, Loss test: 0.1649, Accuracy: 0.9907, Precision: 0.1077, Recall: 0.6020
--------------------------------------------------------------------------------------------------------------
Epoch: 158, Loss train: 0.1630, Loss test: 0.1631, Accuracy: 0.9909, Precision: 0.1097, Recall: 0.6020
--------------------------------------------------------------------------------------------------------------
Epoch: 159, Loss train: 0.1612, Loss test: 0.1613, Accuracy: 0.9912, Precision: 0.1149, Recall: 0.6122
--------------------------------------------------------------------------------------------------------------
Epoch: 160, Loss train: 0.1595, Loss test: 0.1596, Accuracy: 0.9915, Precision: 0.1188, Recall: 0.6122
-----------------------------------------

...

Epoch: 819, Loss train: 0.0060, Loss test: 0.0060, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 820, Loss train: 0.0060, Loss test: 0.0060, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 821, Loss train: 0.0060, Loss test: 0.0060, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 822, Loss train: 0.0060, Loss test: 0.0060, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 823, Loss train: 0.0059, Loss test: 0.0059, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
-----------------------------------------

Epoch: 858, Loss train: 0.0056, Loss test: 0.0057, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 859, Loss train: 0.0056, Loss test: 0.0057, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 860, Loss train: 0.0056, Loss test: 0.0057, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 861, Loss train: 0.0056, Loss test: 0.0057, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 862, Loss train: 0.0056, Loss test: 0.0057, Accuracy: 0.9992, Precision: 0.7732, Recall: 0.7653
-----------------------------------------

Epoch: 897, Loss train: 0.0053, Loss test: 0.0054, Accuracy: 0.9992, Precision: 0.7812, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 898, Loss train: 0.0053, Loss test: 0.0054, Accuracy: 0.9992, Precision: 0.7812, Recall: 0.7653
--------------------------------------------------------------------------------------------------------------
Epoch: 899, Loss train: 0.0053, Loss test: 0.0054, Accuracy: 0.9992, Precision: 0.7835, Recall: 0.7755
--------------------------------------------------------------------------------------------------------------
Epoch: 900, Loss train: 0.0053, Loss test: 0.0054, Accuracy: 0.9992, Precision: 0.7835, Recall: 0.7755
--------------------------------------------------------------------------------------------------------------
Epoch: 901, Loss train: 0.0053, Loss test: 0.0054, Accuracy: 0.9992, Precision: 0.7835, Recall: 0.7755
-----------------------------------------

Epoch: 936, Loss train: 0.0050, Loss test: 0.0052, Accuracy: 0.9993, Precision: 0.7917, Recall: 0.7755
--------------------------------------------------------------------------------------------------------------
Epoch: 937, Loss train: 0.0050, Loss test: 0.0052, Accuracy: 0.9993, Precision: 0.8000, Recall: 0.7755
--------------------------------------------------------------------------------------------------------------
Epoch: 938, Loss train: 0.0050, Loss test: 0.0052, Accuracy: 0.9993, Precision: 0.8000, Recall: 0.7755
--------------------------------------------------------------------------------------------------------------
Epoch: 939, Loss train: 0.0050, Loss test: 0.0052, Accuracy: 0.9993, Precision: 0.8000, Recall: 0.7755
--------------------------------------------------------------------------------------------------------------
Epoch: 940, Loss train: 0.0050, Loss test: 0.0052, Accuracy: 0.9993, Precision: 0.8000, Recall: 0.7755
-----------------------------------------

Epoch: 975, Loss train: 0.0048, Loss test: 0.0050, Accuracy: 0.9993, Precision: 0.8191, Recall: 0.7857
--------------------------------------------------------------------------------------------------------------
Epoch: 976, Loss train: 0.0048, Loss test: 0.0050, Accuracy: 0.9994, Precision: 0.8280, Recall: 0.7857
--------------------------------------------------------------------------------------------------------------
Epoch: 977, Loss train: 0.0048, Loss test: 0.0049, Accuracy: 0.9994, Precision: 0.8280, Recall: 0.7857
--------------------------------------------------------------------------------------------------------------
Epoch: 978, Loss train: 0.0048, Loss test: 0.0050, Accuracy: 0.9994, Precision: 0.8298, Recall: 0.7959
--------------------------------------------------------------------------------------------------------------
Epoch: 979, Loss train: 0.0048, Loss test: 0.0049, Accuracy: 0.9993, Precision: 0.8261, Recall: 0.7755
-----------------------------------------

In [21]:
print(confusion_matrix(actuals,predictions))

[[56848    16]
 [   21    77]]


In [22]:
print(classification_report(actuals,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.79      0.81        98

    accuracy                           1.00     56962
   macro avg       0.91      0.89      0.90     56962
weighted avg       1.00      1.00      1.00     56962

